# Tabular Playground Series - Feb 2022

## Keras - 1 Attempt

## Attempt 1

In [1]:
# 1. Reproducibility
import random
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy
import json
import time

# Set seeds
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# 2. Data Loading, Split & Target Encoding
train_df = pd.read_csv('train.csv.zip')
test_df = pd.read_csv('test.csv.zip')

# Identify id and target columns
id_col = 'row_id' if 'row_id' in train_df.columns else 'id'
target_col = 'target'

df = train_df.copy()
le = LabelEncoder().fit(df[target_col].astype(str))
y_enc = le.transform(df[target_col].astype(str))
classes_ = le.classes_
num_classes = len(classes_)
top_k = min(num_classes, 5)

X = df.drop(columns=[id_col, target_col], errors='ignore')
X_test = test_df.drop(columns=[id_col, target_col], errors='ignore')
test_ids = test_df[id_col]

X_train, X_val, y_train, y_val = train_test_split(
    X, y_enc,
    test_size=0.2,
    stratify=y_enc,
    random_state=seed
)
train_ids = train_df.loc[X_train.index, id_col]
val_ids = train_df.loc[X_val.index, id_col]

# 4. Feature Engineering
missing_cols = [c for c in X_train.columns if X_train[c].isna().all()]
X_train.drop(columns=missing_cols, inplace=True)
X_val.drop(columns=missing_cols, inplace=True)
X_test.drop(columns=missing_cols, inplace=True)

cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
cat_cols = [c for c in cat_cols if X_train[c].nunique() <= 50]
num_cols = [c for c in X_train.columns if c not in cat_cols]

# 5. Preprocessing Pipeline
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)
X_test_proc = preprocessor.transform(X_test)

# 6. Model Architecture
n_samples, n_features = X_train_proc.shape
if n_samples < 10000 or n_features < 100:
    units = [min(n_features*2, 128), min(n_features, 64)]
    drop_rate = 0.3
    use_bn = False
else:
    units = [int(min(n_features*i, 1024)) for i in [2,1,0.5,0.25] if min(n_features*i, 1024) >= 16]
    drop_rate = 0.4
    use_bn = True

inputs = tf.keras.Input(shape=(n_features,))
x = inputs
for u in units:
    x = tf.keras.layers.Dense(u, activation='relu')(x)
    if use_bn:
        x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(drop_rate)(x)
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

# 7. Compile
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', SparseTopKCategoricalAccuracy(k=top_k, name=f'top_{top_k}_accuracy')]
)

# 8. Callbacks & Training
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
]
start_time = time.time()
history = model.fit(
    X_train_proc, y_train,
    validation_data=(X_val_proc, y_val),
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=2
)
duration = time.time() - start_time

# 9. Evaluation & Logging
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# 10. Prediction & Submission
raw_preds = model.predict(X_test_proc)
idxs = raw_preds.argmax(axis=1)
final = le.inverse_transform(idxs)
submission = pd.DataFrame({id_col: test_ids.reset_index(drop=True), target_col: final})
submission.to_csv('submission_result.csv', index=False)

2025-07-10 13:37:50.096073: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752154670.112678 1713711 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752154670.117730 1713711 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752154670.132360 1713711 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752154670.132372 1713711 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752154670.132374 1713711 computation_placer.cc:177] computation placer alr

Epoch 1/100


I0000 00:00:1752154699.302901 1713886 service.cc:152] XLA service 0x740364006130 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752154699.302931 1713886 service.cc:160]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB, Compute Capability 8.0
2025-07-10 13:38:19.396819: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1752154699.726603 1713886 cuda_dnn.cc:529] Loaded cuDNN version 90501
I0000 00:00:1752154703.673110 1713886 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Epoch 1: val_loss improved from inf to 0.30004, saving model to best_model.h5


2500/2500 - 18s - 7ms/step - accuracy: 0.7449 - loss: 0.7031 - top_5_accuracy: 0.9691 - val_accuracy: 0.8866 - val_loss: 0.3000 - val_top_5_accuracy: 0.9903
Epoch 2/100

Epoch 2: val_loss improved from 0.30004 to 0.21361, saving model to best_model.h5


...

2500/2500 - 11s - 4ms/step - accuracy: 0.9815 - loss: 0.0574 - top_5_accuracy: 0.9995 - val_accuracy: 0.9870 - val_loss: 0.0409 - val_top_5_accuracy: 0.9994
Epoch 77/100

Epoch 77: val_loss did not improve from 0.04085
2500/2500 - 12s - 5ms/step - accuracy: 0.9815 - loss: 0.0557 - top_5_accuracy: 0.9995 - val_accuracy: 0.9864 - val_loss: 0.0441 - val_top_5_accuracy: 0.9992
Epoch 78/100

Epoch 78: val_loss did not improve from 0.04085
2500/2500 - 12s - 5ms/step - accuracy: 0.9825 - loss: 0.0543 - top_5_accuracy: 0.9995 - val_accuracy: 0.9863 - val_loss: 0.0433 - val_top_5_accuracy: 0.9995
Epoch 79/100

Epoch 79: val_loss improved from 0.04085 to 0.04030, saving model to best_model.h5


2500/2500 - 10s - 4ms/step - accuracy: 0.9823 - loss: 0.0546 - top_5_accuracy: 0.9995 - val_accuracy: 0.9874 - val_loss: 0.0403 - val_top_5_accuracy: 0.9996
Epoch 80/100

Epoch 80: val_loss did not improve from 0.04030
2500/2500 - 10s - 4ms/step - accuracy: 0.9825 - loss: 0.0541 - top_5_accuracy: 0.9995 - val_accuracy: 0.9873 - val_loss: 0.0408 - val_top_5_accuracy: 0.9996
Epoch 81/100

Epoch 81: val_loss did not improve from 0.04030
2500/2500 - 10s - 4ms/step - accuracy: 0.9825 - loss: 0.0546 - top_5_accuracy: 0.9994 - val_accuracy: 0.9862 - val_loss: 0.0423 - val_top_5_accuracy: 0.9995
Epoch 82/100

Epoch 82: val_loss improved from 0.04030 to 0.03939, saving model to best_model.h5


2500/2500 - 10s - 4ms/step - accuracy: 0.9826 - loss: 0.0537 - top_5_accuracy: 0.9995 - val_accuracy: 0.9871 - val_loss: 0.0394 - val_top_5_accuracy: 0.9994
Epoch 83/100

Epoch 83: val_loss did not improve from 0.03939
2500/2500 - 10s - 4ms/step - accuracy: 0.9826 - loss: 0.0536 - top_5_accuracy: 0.9995 - val_accuracy: 0.9870 - val_loss: 0.0406 - val_top_5_accuracy: 0.9994
Epoch 84/100

Epoch 84: val_loss did not improve from 0.03939
2500/2500 - 10s - 4ms/step - accuracy: 0.9830 - loss: 0.0533 - top_5_accuracy: 0.9996 - val_accuracy: 0.9872 - val_loss: 0.0409 - val_top_5_accuracy: 0.9997
Epoch 85/100

Epoch 85: val_loss improved from 0.03939 to 0.03783, saving model to best_model.h5


2500/2500 - 11s - 4ms/step - accuracy: 0.9832 - loss: 0.0517 - top_5_accuracy: 0.9996 - val_accuracy: 0.9882 - val_loss: 0.0378 - val_top_5_accuracy: 0.9995
Epoch 86/100

Epoch 86: val_loss did not improve from 0.03783
2500/2500 - 11s - 4ms/step - accuracy: 0.9834 - loss: 0.0511 - top_5_accuracy: 0.9995 - val_accuracy: 0.9871 - val_loss: 0.0411 - val_top_5_accuracy: 0.9996
Epoch 87/100

Epoch 87: val_loss did not improve from 0.03783
2500/2500 - 11s - 4ms/step - accuracy: 0.9837 - loss: 0.0505 - top_5_accuracy: 0.9996 - val_accuracy: 0.9805 - val_loss: 0.0623 - val_top_5_accuracy: 0.9995
Epoch 88/100

Epoch 88: val_loss did not improve from 0.03783
2500/2500 - 10s - 4ms/step - accuracy: 0.9839 - loss: 0.0494 - top_5_accuracy: 0.9996 - val_accuracy: 0.9875 - val_loss: 0.0394 - val_top_5_accuracy: 0.9997
Epoch 89/100

Epoch 89: val_loss did not improve from 0.03783
2500/2500 - 10s - 4ms/step - accuracy: 0.9839 - loss: 0.0493 - top_5_accuracy: 0.9996 - val_accuracy: 0.9873 - val_loss: 0.0

2500/2500 - 12s - 5ms/step - accuracy: 0.9843 - loss: 0.0491 - top_5_accuracy: 0.9996 - val_accuracy: 0.9890 - val_loss: 0.0363 - val_top_5_accuracy: 0.9996
Epoch 91/100

Epoch 91: val_loss did not improve from 0.03629
2500/2500 - 11s - 5ms/step - accuracy: 0.9844 - loss: 0.0480 - top_5_accuracy: 0.9997 - val_accuracy: 0.9880 - val_loss: 0.0392 - val_top_5_accuracy: 0.9995
Epoch 92/100

Epoch 92: val_loss did not improve from 0.03629
2500/2500 - 12s - 5ms/step - accuracy: 0.9840 - loss: 0.0491 - top_5_accuracy: 0.9996 - val_accuracy: 0.9858 - val_loss: 0.0448 - val_top_5_accuracy: 0.9995
Epoch 93/100

Epoch 93: val_loss did not improve from 0.03629
2500/2500 - 10s - 4ms/step - accuracy: 0.9848 - loss: 0.0484 - top_5_accuracy: 0.9996 - val_accuracy: 0.9821 - val_loss: 0.0517 - val_top_5_accuracy: 0.9995
Epoch 94/100

Epoch 94: val_loss did not improve from 0.03629
2500/2500 - 11s - 4ms/step - accuracy: 0.9842 - loss: 0.0501 - top_5_accuracy: 0.9996 - val_accuracy: 0.9876 - val_loss: 0.0

In [2]:
print(duration)

1088.3354680538177


## Keras Tuner - 1 Attempt

## Attempt 1

In [ ]:
# 1. Reproducibility
import random
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy
import json
import time

# Set seeds
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# 2. Data Loading, Split & Target Encoding
train_df = pd.read_csv('train.csv.zip')
test_df = pd.read_csv('test.csv.zip')

# Identify id and target columns
id_col = 'row_id' if 'row_id' in train_df.columns else 'id'
target_col = 'target'

df = train_df.copy()
le = LabelEncoder().fit(df[target_col].astype(str))
y_enc = le.transform(df[target_col].astype(str))
classes_ = le.classes_
num_classes = len(classes_)
top_k = min(num_classes, 5)

X = df.drop(columns=[id_col, target_col], errors='ignore')
X_test = test_df.drop(columns=[id_col, target_col], errors='ignore')
test_ids = test_df[id_col]

X_train, X_val, y_train, y_val = train_test_split(
    X, y_enc,
    test_size=0.2,
    stratify=y_enc,
    random_state=seed
)
train_ids = train_df.loc[X_train.index, id_col]
val_ids = train_df.loc[X_val.index, id_col]

# 4. Feature Engineering
missing_cols = [c for c in X_train.columns if X_train[c].isna().all()]
X_train.drop(columns=missing_cols, inplace=True)
X_val.drop(columns=missing_cols, inplace=True)
X_test.drop(columns=missing_cols, inplace=True)

cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
cat_cols = [c for c in cat_cols if X_train[c].nunique() <= 50]
num_cols = [c for c in X_train.columns if c not in cat_cols]

# 5. Preprocessing Pipeline
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)
X_test_proc = preprocessor.transform(X_test)

# 6. Model Architecture
import keras_tuner as kt

# Define early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Input dimension
n_features = X_train_proc.shape[1]

# HyperModel
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024, step=64)
        drop = hp.Float('dropout', 0.0, 0.5, step=0.1)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        inputs = tf.keras.Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = tf.keras.layers.Dense(units, activation='relu')(x)
            x = tf.keras.layers.Dropout(drop)(x)
        outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
        model = tf.keras.Model(inputs, outputs)
        model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model

# Tuner setup
bs = 32  # batch size
ep = 20  # epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=True,
    project_name='bayesian_tuner'
)

# Search for the best hyperparameters
if y_val is not None:
    tuner.search(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )
else:
    tuner.search(
        X_train_proc, y_train,
        validation_split=0.2,
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )

# Build the best model
model = tuner.hypermodel.build(tuner.get_best_hyperparameters(1)[0])

start_time = time.time()  # Start timing

# Retrain the model with original callbacks and data
if y_val is not None:
    history = model.fit(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
else:
    history = model.fit(
        X_train_proc, y_train,
        validation_split=0.2,
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )

duration = time.time() - start_time  # Calculate duration

# 9. Evaluation & Logging
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# 10. Prediction & Submission
raw_preds = model.predict(X_test_proc)
idxs = raw_preds.argmax(axis=1)
final = le.inverse_transform(idxs)
submission = pd.DataFrame({id_col: test_ids.reset_index(drop=True), target_col: final})
submission.to_csv('submission_result.csv', index=False)

Trial 10 Complete [00h 07m 19s]
val_loss: 0.076991006731987

Best val_loss So Far: 0.076991006731987
Total elapsed time: 01h 14m 38s
Epoch 1/100


5000/5000 - 23s - 5ms/step - accuracy: 0.8891 - loss: 0.3001 - val_accuracy: 0.9351 - val_loss: 0.1746
Epoch 2/100


5000/5000 - 19s - 4ms/step - accuracy: 0.9427 - loss: 0.1572 - val_accuracy: 0.9518 - val_loss: 0.1404
Epoch 3/100


5000/5000 - 18s - 4ms/step - accuracy: 0.9550 - loss: 0.1255 - val_accuracy: 0.9509 - val_loss: 0.1402
Epoch 4/100


5000/5000 - 18s - 4ms/step - accuracy: 0.9606 - loss: 0.1120 - val_accuracy: 0.9489 - val_loss: 0.1289
Epoch 5/100


...

5000/5000 - 21s - 4ms/step - accuracy: 0.9720 - loss: 0.0846 - val_accuracy: 0.9693 - val_loss: 0.0917
Epoch 8/100
5000/5000 - 21s - 4ms/step - accuracy: 0.9740 - loss: 0.0804 - val_accuracy: 0.9689 - val_loss: 0.0940
Epoch 9/100
5000/5000 - 21s - 4ms/step - accuracy: 0.9757 - loss: 0.0734 - val_accuracy: 0.9660 - val_loss: 0.1004
Epoch 10/100
5000/5000 - 20s - 4ms/step - accuracy: 0.9778 - loss: 0.0671 - val_accuracy: 0.9675 - val_loss: 0.0975
Epoch 11/100


5000/5000 - 19s - 4ms/step - accuracy: 0.9787 - loss: 0.0682 - val_accuracy: 0.9747 - val_loss: 0.0899
Epoch 12/100
5000/5000 - 19s - 4ms/step - accuracy: 0.9802 - loss: 0.0657 - val_accuracy: 0.9734 - val_loss: 0.0946
Epoch 13/100


5000/5000 - 18s - 4ms/step - accuracy: 0.9804 - loss: 0.0637 - val_accuracy: 0.9758 - val_loss: 0.0839
Epoch 14/100
5000/5000 - 19s - 4ms/step - accuracy: 0.9818 - loss: 0.0644 - val_accuracy: 0.9738 - val_loss: 0.1031
Epoch 15/100
5000/5000 - 19s - 4ms/step - accuracy: 0.9833 - loss: 0.0532 - val_accuracy: 0.9764 - val_loss: 0.0862
Epoch 16/100
5000/5000 - 19s - 4ms/step - accuracy: 0.9830 - loss: 0.0544 - val_accuracy: 0.9766 - val_loss: 0.0860
Epoch 17/100


5000/5000 - 20s - 4ms/step - accuracy: 0.9843 - loss: 0.0552 - val_accuracy: 0.9809 - val_loss: 0.0720
Epoch 18/100
5000/5000 - 21s - 4ms/step - accuracy: 0.9850 - loss: 0.0490 - val_accuracy: 0.9796 - val_loss: 0.0890
Epoch 19/100
5000/5000 - 23s - 5ms/step - accuracy: 0.9849 - loss: 0.0557 - val_accuracy: 0.9800 - val_loss: 0.0793
Epoch 20/100
5000/5000 - 22s - 4ms/step - accuracy: 0.9858 - loss: 0.0480 - val_accuracy: 0.9805 - val_loss: 0.1034
Epoch 21/100
5000/5000 - 22s - 4ms/step - accuracy: 0.9864 - loss: 0.0492 - val_accuracy: 0.9806 - val_loss: 0.0941
Epoch 22/100
5000/5000 - 21s - 4ms/step - accuracy: 0.9862 - loss: 0.0472 - val_accuracy: 0.9757 - val_loss: 0.1197
Epoch 23/100
5000/5000 - 18s - 4ms/step - accuracy: 0.9879 - loss: 0.0441 - val_accuracy: 0.9811 - val_loss: 0.0749
Epoch 24/100
5000/5000 - 18s - 4ms/step - accuracy: 0.9872 - loss: 0.0543 - val_accuracy: 0.9811 - val_loss: 0.1243
Epoch 25/100
5000/5000 - 18s - 4ms/step - accuracy: 0.9886 - loss: 0.0451 - val_accur

5000/5000 - 19s - 4ms/step - accuracy: 0.9893 - loss: 0.0396 - val_accuracy: 0.9862 - val_loss: 0.0659
Epoch 28/100
5000/5000 - 18s - 4ms/step - accuracy: 0.9895 - loss: 0.0427 - val_accuracy: 0.9832 - val_loss: 0.0840
Epoch 29/100
5000/5000 - 19s - 4ms/step - accuracy: 0.9890 - loss: 0.0481 - val_accuracy: 0.9808 - val_loss: 0.1194
Epoch 30/100
5000/5000 - 19s - 4ms/step - accuracy: 0.9898 - loss: 0.0402 - val_accuracy: 0.9778 - val_loss: 0.0988
Epoch 31/100
5000/5000 - 18s - 4ms/step - accuracy: 0.9897 - loss: 0.0446 - val_accuracy: 0.9837 - val_loss: 0.0730
Epoch 32/100
5000/5000 - 20s - 4ms/step - accuracy: 0.9903 - loss: 0.0398 - val_accuracy: 0.9851 - val_loss: 0.0821
Epoch 33/100
5000/5000 - 19s - 4ms/step - accuracy: 0.9907 - loss: 0.0401 - val_accuracy: 0.9864 - val_loss: 0.0750
Epoch 34/100
5000/5000 - 22s - 4ms/step - accuracy: 0.9905 - loss: 0.0412 - val_accuracy: 0.9827 - val_loss: 0.0852
Epoch 35/100
5000/5000 - 21s - 4ms/step - accuracy: 0.9904 - loss: 0.0395 - val_accur

In [ ]:
print(duration)

732.146827
